## Object Detection Model 비교

- R-CNN 계열들은 높은 정밀도를 가지지만, FPS가 받쳐주지 못함
- 2 Stage Detector에서 -> 1 Stage Detector로 가면서 YOLO, YOLO V2, SSD 등이 등장

![object_detection_flow](img/object_detection_flow.PNG)

## YOLO 문제점 및 한계

![SSD_VS_YOLO](img/SSD_VS_YOLO.png)

- Yolo는 속도를 Faster R-CNN(7FPS) -> YOLO(45FPS)까지 올리는 비약적인 발전을 이룸
- 하지만, YOLO의 7x7 그리드 크기보다 작은 물체들은 잘 잡아내지 못한다는 한계점 존재
- 신경망을 모두 통과하며 컨볼루션과 풀링을 거치다보니 Coarse한 마지막 단 피쳐맵만 사용하여 정확도 하락
- **YOLO보다 성능도 좋고 속도도 빠름**
    - FC Layer 안쓰고 Conv만 이용. 
    - 여러 Feature map과 다양한 grid로 접근 -> 다양한 크기의 물체 detect 가능
<hr>


## SSD 특징 - Multi Scale Feature Maps for Detection
- SSD: Single Shot MultiBox Detector
- SSD는 Yolo와 달리 컨볼루션 과정을 거치는 중간 중간 피쳐맵들에서 모두 Object Detection을 진행함
- SSD는 input **300x300 크기의 이미지를 입력 및 ImageNet pretrained된 VGG-16 통과(VGG-16 conv4_3까지 적용, base Network) ==> output : 38x38x512 부터 시작**

![SSD_Boundingbox](img/SSD_Boundingbox.png)


## SSD Feature Map
- **컨볼루션을 거치면서 디테일한 정보들이 사라지는 문제점을 앞단의 피쳐맵들을 끌어오는 방식으로 해결**
- 38x38, 19x19, 10x10, 3x3, 1x1의 피쳐맵들은 output과 직결되는 피쳐맵임
- 각 피쳐맵별 적절한 conv연산을 통해 bounding box의 class점수와 offset을 얻게됨.
- conv filter size => 3 x 3 x (바운딩박스 개수 x (class score + offset))
- 이 특징맵들의 크기는 모두 다르며, 마지막 conv11_2 를 제외하고는 5개의 특징맵은 2번 컨볼루션한다.

- **최종적으로는 8732x(Classes+4)로 클래스당 8732개의 경계박스 예측**
    - 38x38x(4x(Classes+4)) = 5776x(Classes+4) : Feature 갯수임
    - 19x19x(6x(Classes+4)) = 2166x(Classes+4)
    - 10x10x(6x(Classes+4)) = 600x(Classes+4)
    - 5x5x(6x(Classes+4)) = 150x(Classes+4)
    - 3x3x(4x(Classes+4)) = 36x(Classes+4)
    - 1x1x(4x(Classes+4)) = 4x(Classes+4)

- **8732개의 Boundingbox output중(최종 6개의 Classifier의 예측 중) box의 IOU가 0.5이상이 되는 box들만 고려대상에 포함 (NMS)**
- **큰 Featuremap에서는 작은 물체를 탐지, 작은 Feature map에서는 큰 물체 탐지**

![SSD_Boundingbox2](img/SSD_Boundingbox2.png)


<hr>

## SSD 특징 - Hard Negative Mining
- Matching 단계가 지난 후 대부분의 Default box들은 negative(배경)임
- 이것은 positive와 negative 학습 데이터의 언밸런스 문제로 이어지며 해당 비율을 (negative)3:1(positive)로 사용.

![SSD_Hard_Negative_Mining](img/SSD_Hard_Negative_Mining.PNG)

<hr>


## Loss Function
- 전체적으로 로스는 Yolo의 로스 펑션과 매우 유사
- 전체 로스는 각 클래스 별로 예측한 값과 실제 값 사이의 차인 Lconf와 바운딩 박스 리그레션 예측 값과 실제 값 사이의 차인 Lloc를 더한 값

![SSD_LossFunction2](img/SSD_LossFunction2.png)

<hr>

## 결론
- multiple categories 1-stage detector
- Data Augmentation 성능 8% 증가
- 많은 Default box모양을 사용
    - 많은 종류의 box사용시 더 box를 잘예측함
- **층을 거치며 여러 개의 featuremap으로부터 다양한 크기의 bounding box를 가져오는 것**


## 참고문헌
- https://taeu.github.io/paper/deeplearning-paper-ssd/ // 설명 잘나와있음
- https://yeomko.tistory.com/20
- https://hohodu.tistory.com/8

